In [1]:
from hypothesis import given, strategies as st

In [2]:
# import ipytest
# ipytest.autoconfig()
# ipytest.config(clean=False)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [4]:
from tqdm import tqdm, tqdm_notebook

from dataclasses import dataclass
from typing import *

from copy import deepcopy
from functools import reduce
from itertools import product

# `funcy` provides some convenient functionally-flavored combinators for manipulating collections and 
# dictionaries used throughout: https://funcy.readthedocs.io/en/stable/index.html
from funcy import * 

# from z3 import *
import z3

#import igraph as ig
#import pygraphviz as pgv
#from IPython.display import Image

In [5]:
def block_current_model(s):
  assert s.check() == z3.sat, f"no model to block!"
  m = s.model()
  if len([f for f in m.decls() if f.arity() == 0]) > 0:
    s.add(z3.Or([ f() != m[f] for f in m.decls() if f.arity() == 0]))
  return s

def all_smt(s):
  models = []
  while z3.sat == s.check():
    models.append(s.model())
    print(f"model # {len(models)}")
    #print(s.model())
    block_current_model(s)
  return models

def block_current_model_just(s, terms):
  assert s.check() == z3.sat, f"no model to block!"
  m = s.model()
  s.add(z3.Or([t != m.eval(t, model_completion=True) for t in terms]))
  return s

def all_smt_just(s, terms):
  models = []
  while z3.sat == s.check():
    models.append(s.model())
    print(f"model # {len(models)}")
    #print(s.model())
    block_current_model_just(s, terms)
  return models

In [6]:
from FV import *

# Confirm concrete and symbolic (pair of parallel Boolean list encoding) feature vectors have matching behavior

In [7]:
#%%ipytest

def test_concreteSymbolic_match_join():
  joinCounterexamples = testBinOp_Booleans(SymbolicFV.join, FeatureVector.join, 3, exceptions=True)
  len(joinCounterexamples)

  for i,x in tqdm(enumerate(joinCounterexamples), total=len(joinCounterexamples)):
    print(f"{i}: " + showBinGraphTest("v", x[0], x[1], x[2], x[3]))
    
  assert len(joinCounterexamples) == 0

EE.                                                                                          [100%]
============================================== ERRORS ==============================================
_______________________________ ERROR at setup of testBinOp_Booleans _______________________________
file /home/emeinhardt/Babel/chromatic-cactus/FV.py, line 751
  def testBinOp_Booleans(binOpRel, binOp, numFeatures, testSet=None, exceptions=False, none=False):
E       fixture 'binOpRel' not found
>       available fixtures: cache, capfd, capfdbinary, caplog, capsys, capsysbinary, doctest_namespace, monkeypatch, pytestconfig, record_property, record_testsuite_property, record_xml_attribute, recwarn, tmp_path, tmp_path_factory, tmpdir, tmpdir_factory
>       use 'pytest --fixtures [testpath]' for help on them.

/home/emeinhardt/Babel/chromatic-cactus/FV.py:751
______________________________ ERROR at setup of testStackOp_Booleans ______________________________
file /home/emeinhardt/Babel/ch

In [28]:
def test_concreteSymbolic_match_meet():
  meetCounterexamples = testBinOp_Booleans(SymbolicFV.meet, FeatureVector.meet, 3, exceptions=False)
  len(meetCounterexamples)

  for i,x in tqdm(enumerate(meetCounterexamples), total=len(meetCounterexamples)):
    print(f"{i}: " + showBinGraphTest("v", x[0], x[1], x[2], x[3]))
    
  assert len(meetCounteExamples) == 0

In [29]:
test_concreteSymbolic_match_join()

0it [00:00, ?it/s]


In [30]:
test_concreteSymbolic_match_meet()

0it [00:00, ?it/s]


# Define a concrete feature system and inventory to work with

## Generate a subset $S$ of at least $k$ objects from a set with at least $n$ objects over $m$ features such that $spec(meet(S)) \geq i$

In [9]:
M = 3
MAX_OBJECT_SET_SIZE = 26
# M = 10
# MAX_OBJECT_SET_SIZE = 100
MAX_SAMPLE_SIZE = 8
# MAX_OBJECT_SET_SIZE = MAX_NUM_VECTORS
# MAX_SAMPLE_SIZE = MAX_NUM_VECTORS 
MAX_NUM_VECTORS = 3 ** M
N = randint(20, MAX_OBJECT_SET_SIZE)
assert N <= (3 ** M)
print(f"|features| = {M}")
print(f"=> |logically possible vectors| = {MAX_NUM_VECTORS}")
print(f"max |objects = inventory| = {MAX_OBJECT_SET_SIZE}")
print(f"max |sample = hypothetical natural class| = {MAX_SAMPLE_SIZE}")
print(f"actual |objects = inventory| = {N}")

|features| = 3
=> |logically possible vectors| = 27
max |objects = inventory| = 26
max |sample = hypothetical natural class| = 8
actual |objects = inventory| = 22


In [10]:
O = FeatureVector.randomObjects(numFeatures=M, numObjects=N)

if len(O) < 30:
  for each in O:
    print(each)

[0 0 0]
[0 + 0]
[0 0 -]
[0 0 +]
[0 - 0]
[- 0 0]
[0 - -]
[- 0 +]
[0 - +]
[+ - 0]
[- - 0]
[0 + +]
[+ 0 -]
[- + 0]
[- + -]
[- + +]
[+ - +]
[+ - -]
[- - -]
[- - +]
[+ + -]
[+ + +]


In [11]:
# S = randomNonemptySubset(set(O), 3)
S = randomInterestingSubset(200000, set(O), cap=MAX_SAMPLE_SIZE, minSpec=2) # "interesting" = the natural class is something other the trivial zero vector [0 ... ]

if S is not None:
  for each in S:
    print(each)

  print("---")
  print(grandFoldl(FeatureVector.meet)(S))
else:
  print("S is None.")

[0 - -]
[+ - -]
---
[0 - -]


# Generate a set of $n$ objects $O$ over $m$ features such that the meet is not unique

In [31]:
def findObjectSetWithNonUniqueMeet(numFeatures, numObjects):
  assert numFeatures > 0
  assert numObjects  > 0
  maxNumVectors = 3 ** numFeatures
  assert numObjects <= maxNumVectors, f"Max # distinct vectors for {numFeatures} features is {maxNumVectors}, but numObjects is '{numObjects}'"
  
  s = z3.Solver()
  
  stack = [SymbolicFV.mk(numFeatures, f"V_{i}") for i in range(numObjects)]
  defineds, valences = SymbolicFV.unwraps(tuple)(stack)
  s.add(SymbolicFV.pairwise_distinct1(stack))
  
  meet = SymbolicFV.mk(numFeatures, "m")
  defined_meet, valence_meet = meet.unwrap()
  s.add(SymbolicFV.meet1(stack, meet))
  
  alt = SymbolicFV.mk(numFeatures, "a")
  defined_alt , valence_alt  = alt.unwrap()
  s.add(SymbolicFV.meet1(stack, alt))
  
  s.push()
  s.add(SymbolicFV.distinct(meet, alt))

  if s.check() == z3.sat:
    print('Found model!?') # not expected!
    return "nonunique meets", s, s.model(), stack, meet, alt
  else:
    print('No model with distinct second meet') # evidence our definition of meet_Booleans1 is correct if this is *always* the outcome
    s.pop()
    if s.check() == z3.sat:
      print('unique meet')
      return "unique meet/popped", s, s.model(), stack, meet, alt
    print('No meet!')
    return "no meet", s

In [32]:
def test_searchFor_objectSetsWithNonuniqueMeet():
  Ms =  list(range(3,5))
  Ns = [list(range(1, 3 ** m)) for m in Ms]
  MNs = lmapcat(lambda pair: [(pair[0], each) for each in pair[1]], 
                zip(Ms, Ns))
  meetIsUnique, meetIsNotUnique, noMeet = [], [], []
  for eachM, eachN in tqdm(MNs, total = len(MNs)):
    print(f"m = {eachM}")
    print(f"n = {eachN}")
    res = findObjectSetWithNonUniqueMeet(eachM, eachN)
    if res[0] == "nonunique meets":
      meetIsNotUnique.append(res)
    elif res[0] == "no meet":
      noMeet.append(res)
    elif res[0] == "unique meet/popped":
      meetIsUnique.append(res)
  print(f"|models where meet is *not* unique| = {len(meetIsNotUnique)}") # expecting zero
  print(f"|models where no meet exists| = {len(noMeet)}") # expecting zero
  print(f"|models where meet is unique| = {len(meetIsUnique)}")
  
  assert len(meetIsNotUnique) == 0
  
  return meetIsUnique, meetIsNotUnique, noMeet



In [33]:
results = searchFor_objectSetsWithNonuniqueMeet()

  5%|▍         | 5/106 [00:00<00:02, 35.88it/s]

m = 3
n = 1
No model with distinct second meet
unique meet
m = 3
n = 2
No model with distinct second meet
unique meet
m = 3
n = 3
No model with distinct second meet
unique meet
m = 3
n = 4
No model with distinct second meet
unique meet
m = 3
n = 5
No model with distinct second meet
unique meet
m = 3
n = 6
No model with distinct second meet
unique meet
m = 3
n = 7


  8%|▊         | 9/106 [00:00<00:05, 18.77it/s]

No model with distinct second meet
unique meet
m = 3
n = 8
No model with distinct second meet
unique meet
m = 3
n = 9
No model with distinct second meet
unique meet
m = 3
n = 10
No model with distinct second meet
unique meet
m = 3
n = 11
No model with distinct second meet
unique meet
m = 3
n = 12


 11%|█▏        | 12/106 [00:00<00:08, 11.66it/s]

...

unique meet
m = 4
n = 79
No model with distinct second meet


 99%|█████████▉| 105/106 [03:46<00:08,  8.02s/it]

unique meet
m = 4
n = 80
No model with distinct second meet


100%|██████████| 106/106 [03:54<00:00,  2.21s/it]

unique meet
|models where meet is *not* unique| = 0
|models where no meet exists| = 0
|models where meet is unique| = 106


## (...join analogue)

In [14]:
def findObjectSetWithNonUniqueJoin(numFeatures, numObjects):
  assert numFeatures > 0
  assert numObjects  > 0
  maxNumVectors = 3 ** numFeatures
  assert numObjects <= maxNumVectors, f"Max # distinct vectors for {numFeatures} features is {maxNumVectors}, but numObjects is '{numObjects}'"
  
  s = z3.Solver()
  
  stack = [SymbolicFV.mk(numFeatures, f"V_{i}") for i in range(numObjects)]
  defineds, valences = SymbolicFV.unwraps(tuple)(stack)
  s.add(SymbolicFV.pairwise_distinct1(stack))
  
  join = SymbolicFV.mk(numFeatures, "j")
  defined_join, valence_join = join.unwrap()
  s.add(SymbolicFV.meet1(stack, join))
  
  alt = SymbolicFV.mk(numFeatures, "a")
  defined_alt , valence_alt  = alt.unwrap()
  s.add(SymbolicFV.meet1(stack, alt))
  
  s.push()
  s.add(SymbolicFV.distinct(join, alt))

  if s.check() == z3.sat:
    print('Found model!?') # not expected!
    return "nonunique joins", s, s.model(), stack, join, alt
  else:
    print('No model with distinct second join') # evidence our definition of join_Booleans1 is correct if this is *always* the outcome
    s.pop()
    if s.check() == z3.sat:
      print('unique join')
      return "unique join/popped", s, s.model(), stack, join, alt
    print('No join!')
    return "no join", s

In [15]:
#def test_searchFor_objectSetsWithNonuniqueJoin():
def searchFor_objectSetsWithNonuniqueJoin():
  Ms =  list(range(3,5))
  Ns = [list(range(1, 3 ** m)) for m in Ms]
  MNs = lmapcat(lambda pair: [(pair[0], each) for each in pair[1]], 
                zip(Ms, Ns))
  joinIsUnique, joinIsNotUnique, noJoin = [], [], []
  for eachM, eachN in tqdm(MNs, total = len(MNs)):
    print(f"m = {eachM}")
    print(f"n = {eachN}")
    res = findObjectSetWithNonUniqueJoin(eachM, eachN)
    if res[0] == "nonunique joins":
      joinIsNotUnique.append(res)
    elif res[0] == "no join":
      noJoin.append(res)
    elif res[0] == "unique join/popped":
      joinIsUnique.append(res)
  print(f"|models where join is *not* unique| = {len(joinIsNotUnique)}") # expecting zero
  print(f"|models where no join exists| = {len(noJoin)}") # expecting *non*zero, perhaps most
  print(f"|models where join is unique| = {len(joinIsUnique)}")
  
  assert len(joinIsNotUnique) == 0
  
  return joinIsUnique, joinIsNotUnique, noJoin


In [ ]:
results = searchFor_objectSetsWithNonuniqueJoin()

## Generate a set of $n$ objects $O$ over $m$ features such that there is a subset $S$ whose meet $m$ has *synonyms* - feature vectors with the same interpretation

In [16]:
def findAmbiguousObjectSet(numFeatures, numObjects, 
                           minSubsetSize=None, 
                           meetIsNonZero=True, altIsNonZero=True, 
                           interpIsNotUniverse=True,
                           par=True, solver=None):
  if solver is None:
    solver = z3.Solver()
  if par:
    z3.set_param("parallel.enable", True)
#     z3.set_param("parallel.threads.max", 4) # defaults to number of processes
  if minSubsetSize is None:
    minSubsetSize = 2
  assert minSubsetSize <= numObjects
    
  stack = SymbolicFV.mks(numFeatures, "V", numObjects)
  defineds, valences = SymbolicFV.unwraps(tuple)(stack)
  solver.add(SymbolicFV.pairwise_distinct1(stack))
  
  inSubset = SymbolicSubsetVector.mk(numObjects, "inSubset")
#   solver.add(SymbolicSubsetVector.isNonempty(inSubset))
  solver.add(SymbolicSubsetVector.hasAtLeastKelements(inSubset, minSubsetSize))
  
  meet = SymbolicFV.mk(numFeatures, "meet")
  defined_meet, valence_meet = meet.unwrap()
  solver.add(SymbolicFV.meet2(stack, inSubset, meet))
  if meetIsNonZero:
    solver.add(SymbolicFV.nonZero(meet))
  
  # assert that every object in the subset must be in the interpretation of the meet (the converse does not in general hold)
  # assert that for every object i, i is in the interpretation of the meet iff the meet <= object i 
  meet_interp = SymbolicSubsetVector.mk(numObjects, "meet_interp")
  solver.add(SymbolicFV.interpretation(meet, meet_interp, stack))
  solver.add(z3.And([z3.Implies(inSubset.value[i], meet_interp.value[i])
                    for i in range(numObjects)]))
  if interpIsNotUniverse:
    solver.add(z3.Not(SymbolicSubsetVector.isUniverse(meet_interp)))
  
  alt = SymbolicFV.mk(numFeatures, "alt")
  defined_alt, valence_alt = alt.unwrap()

  solver.add(SymbolicFV.distinct(meet, alt))
  if altIsNonZero:
    solver.add(SymbolicFV.nonZero(alt))
  
  alt_interp = SymbolicSubsetVector.mk(numObjects, "alt_interp")
  solver.add(SymbolicFV.interpretation(alt, alt_interp, stack))

  solver.push()
  solver.add(z3.And([m == a 
                     for m,a in zip(meet_interp.value, 
                                    alt_interp.value)
                    ]))
  
  if solver.check() == z3.sat:
    print("Found vector distinct from meet with same interpretation")
    return "meet synonym", solver, solver.model(), stack, inSubset, meet, meet_interp, alt, alt_interp
  else:
    solver.pop()
    if solver.check() == z3.sat:
      print("Only meet has its interpretation")
      return "unique meet", solver, solver.model(), stack, inSubset, meet, meet_interp, alt, alt_interp
    else:
      print("Meet does not exist with current parameters")
      return "no meet", solver


In [16]:
def searchForAmbiguousObjectSets():
  Ms  = tuple(range(3,4)) # (3,5) ; (4,5) ; (4,12) ; (5,6) ; (5,20); (3,20) ; (3,4)
  MIN_NUM_OBJECTS = 10 # 20 ; 10
  MAX_NUM_OBJECTS = 12 # 28 ; 15 ; 20 ; 15 ; 40 ; 12
  Ns  = tuple([tuple(range(MIN_NUM_OBJECTS, 3 ** m)) if m < 3 else tuple(range(MIN_NUM_OBJECTS, MAX_NUM_OBJECTS))
               for m in Ms])
  MNs = lmapcat(lambda pair: tuple([(pair[0], each) for each in pair[1]]),
                zip(Ms, Ns))
  print(f"|MNs|={len(MNs)}")
  print("MNs:")
  for each in MNs:
    print(f"m={each[0]}, n={each[1]}")
  
  meetSynonyms, uniqueMeets, noMeets = [], [], []
  for eachM, eachN in tqdm(MNs, total = len(MNs)):
    print(f"m = {eachM}")
    print(f"n = {eachN}")
    res = findAmbiguousObjectSet(eachM, eachN, minSubsetSize=2, meetIsNonZero=True, altIsNonZero=True)
    if res[0] == "meet synonym":
      meetSynonyms.append(res)
    elif res[0] == "unique meet":
      uniqueMeets.append(res)
    elif res[0] == "no meet":
      noMeets.append(res)
  print(f"|models where meet has a synonym| = {len(meetSynonyms)}") # expecting non-zero
  print(f"|models where meet is unique| = {len(uniqueMeets)}") # expecting most
  print(f"|models where no meet exists| = {len(noMeets)}") # expecting zero
  return meetSynonyms, uniqueMeets, noMeets

results = searchForAmbiguousObjectSets()

|MNs|=2
MNs:
m=3, n=10
m=3, n=11


  0%|          | 0/2 [00:00<?, ?it/s]


m = 3
n = 10


NameError: name 'findAmbiguousObjectSet' is not defined

In [ ]:
syns = results[0]
len(syns)
len(syns[0])
s0 = syns[0]
s0[2]
s0[3]
s0[4]
s0[5]
s0[6]
s0[7]

In [ ]:
type(s0[1])
type(s0[2])
type(s0[3])

In [ ]:
def extract_ambiguousObjectSetModel(result):
  tag, solver, model, stack, inSubset, meet, meet_interp, alt, alt_interp = result
  return { "tag":     tag
         ,  "solver": solver , "model":       model
         ,  "stack":  stack  , "inSubset":    inSubset
         ,  "meet":   meet   , "meet_interp": meet_interp
         ,  "alt":    alt    , "alt_interp":  alt_interp
         }

def concretize_ambiguousObjectSetModel(extractedResult):
  model        = extractedResult["model"]
  stack        = extractedResult["stack"]
  inSubset     = extractedResult["inSubset"]
  meet         = extractedResult["meet"]
  meet_interp  = extractedResult["meet_interp"]
  alt          = extractedResult["alt"]
  alt_interp   = extractedResult["alt_interp"]

  objects    = SymbolicFV.toConcreteFeatureVectors(stack, model)
  subsetVec  = inSubset.toConcreteSubsetVector(model)
  subset     = [objects[i] for i in range(len(objects)) if subsetVec[i]]
  meet_c     = meet.toConcreteFeatureVector(model)
  alt_c      = alt.toConcreteFeatureVector(model)
  meetInterp = meet_interp.toConcreteSubsetVector(model)
  altInterp  = meet_interp.toConcreteSubsetVector(model)

  return { "objects": objects, "subsetVec":  subsetVec , "subset":    subset, 
           "meet":    meet_c , "meetInterp": meetInterp,
           "alt":     alt_c  , "altInterp":  altInterp
         }

def report_ambiguousObjectSetModel(concretized_result):
  # objects, subsetVec, subset, meet, alt, meetInterp, altInterp = result
  objects    = concretized_result["objects"]
  subsetVec  = concretized_result["subsetVec"]
  subset     = concretized_result["subset"]
  meet       = concretized_result["meet"]
  meetInterp = concretized_result["meetInterp"]
  alt        = concretized_result["alt"]
  altInterp  = concretized_result["altInterp"]


  print(f"|objects| = {len(objects)}")
  print(f"|subset|  = {len(subset)}")
  print('---')
  print(f"subset:")
  for each in subset:
    print(each)
  print('---')
  print(f"meet vs. alt:\n{meet}\n{alt}")
  print(f"meet interp vs. alt interp:\n{meetInterp}\n{altInterp}")

In [ ]:
report_ambiguousObjectSetModel(concretize_ambiguousObjectSetModel(extract_ambiguousObjectSetModel(s0)))

In [ ]:
cyns = lmap(lambda s: concretize_ambiguousObjectSetModel(extract_ambiguousObjectSetModel(s)), syns)

In [ ]:
hasNonemptySubset = []
for c in cyns:
  if len(c["subset"]) > 0:
    hasNonemptySubset.append(c)

len(hasNonemptySubset)

In [ ]:
cyns[0]['objects']

In [ ]:
lzip(cyns[0]['subsetVec'], range(len(cyns[0]['subsetVec'])))
len(cyns[0]['subsetVec'])
for each in cyns[0]['subset']:
  print(each)

In [ ]:
print(cyns[0]['meet'])
lzip(cyns[0]['meetInterp'], range(len(cyns[0]['meetInterp'])))
lzip(cyns[0]['subsetVec'], range(len(cyns[0]['subsetVec'])))

In [ ]:
print(cyns[0]['alt'])
lzip(cyns[0]['altInterp'], range(len(cyns[0]['altInterp'])))
lzip(cyns[0]['subsetVec'], range(len(cyns[0]['subsetVec'])))

In [ ]:
lzip(cyns[0]['meetInterp'], 
     cyns[0]['altInterp'],
     cyns[0]['subsetVec'],
     range(len(cyns[0]['meetInterp'])))

In [ ]:
print(cyns[0]['meet'])
print(cyns[0]['alt'])
for i, each in enumerate(cyns[0]['objects']):
  print(f"{i}: {each}\t{cyns[0]['subsetVec'][i]}\t {cyns[0]['meet'] <= each}\t {cyns[0]['alt'] <= each}")

# Scratch

In [ ]:
#show use of a quantified statement
def quantifierExperimentAdditiveIdentity(x):
  y = z3.Bool('y') # <- note that the way I'm using this, I won't have access to this again
  return z3.ForAll([y], z3.Or(x,y))

def findBooleanDisjunctiveIdentity():
  s = z3.Solver()
  x = z3.Bool('x')
  s.add(quantifierExperimentAdditiveIdentity(x)) 
  if s.check() == z3.sat:
    print("Found disjunctive idenitity!")
    return s, s.model(), x
  else:
    print("Disjunction has no identity: logic is canceled.")
    return s

res = findBooleanDisjunctiveIdentity()

In [ ]:
res[1][res[2]]

In [ ]:
def quantifierExperimentAdditiveIdentity1(x,k):
  Ys = [z3.Bool(f'y_{i}') for i in range(k)]
  return z3.ForAll(Ys, z3.Or([x] + Ys))

def findBooleanDisjunctiveIdentity1(k):
  s = z3.Solver()
  x = z3.Bool('x')
  s.add(quantifierExperimentAdditiveIdentity1(x,k)) 
  if s.check() == z3.sat:
    print("Found disjunctive idenitity!")
    return s, s.model(), x
  else:
    print("Disjunction has no identity: logic is canceled.")
    return s

res1 = findBooleanDisjunctiveIdentity1(k=3)

In [ ]:
res1[0]
res1[1]
res1[1][res[2]]

In [ ]:
def getAllSynonymsFor(concreteFV, concreteFVs_objects):
  ensureSameLengths(concreteFVs_objects)